In [1]:
import org.apache.spark.sql.types.{StructType, StringType}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame


In [2]:
println(s"Current spark version is ${spark.version}")

Current spark version is 2.4.4


In [3]:
val inputStreamPath = "/home/jovyan/data/events-stream"
val modelPath = "/home/jovyan/models/spark-ml-model"

val dataSchema = new StructType()
    .add("tweet", StringType)

val inputDF = spark
    .readStream
    .schema(dataSchema)
    .option("maxFilesPerTrigger", 1)
    .json(inputStreamPath)

inputStreamPath = /home/jovyan/data/events-stream
modelPath = /home/jovyan/models/spark-ml-model
dataSchema = StructType(StructField(tweet,StringType,true))
inputDF = [tweet: string]


[tweet: string]

In [4]:
val model = PipelineModel.load(modelPath)

model = pipeline_f1fbeb79703f


pipeline_f1fbeb79703f

In [8]:
import org.apache.spark.ml.linalg.Vector

val getProbability = udf((prediction: Vector) => prediction(1))

val predictionsDF = model.transform(inputDF)
predictionsDF.writeStream.foreachBatch { (batchDF: DataFrame, batchId: Long) =>
    batchDF.select($"tweet", getProbability($"probability").alias("negative_probability")).show(20, false)
}.start()

+---------------------------------------------------------------------------------------------------------------------------------------+--------------------+
|tweet                                                                                                                                  |negative_probability|
+---------------------------------------------------------------------------------------------------------------------------------------+--------------------+
|where did the sun go??                                                                                                                 |0.5144744106785156  |
|@mackmaine WHY U STOP FOLLOWIN ME?                                                                                                     |0.4948742956877036  |
|@glimpsesofsoul  i want to hammock.                                                                                                    |0.49264848515574655 |
|last day of expression engine #EE training   

getProbability = UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))
predictionsDF = [tweet: string, words: array<string> ... 4 more fields]
stream = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@165d69dd


org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@165d69dd

+------------------------------------------------------------------------------------------------------------------------------------------+--------------------+
|tweet                                                                                                                                     |negative_probability|
+------------------------------------------------------------------------------------------------------------------------------------------+--------------------+
|@kwells2416 Yeah I was busy  How's it going? Big weekend planned?                                                                         |0.5079832589933295  |
|Shaun love, should have done another song  Still amazing though!                                                                          |0.4697099537036154  |
|@pcassle I respect opinion and I'm disappointed you needed to vent further, I recooked,visited table, and bought the steak? why this now? |0.5471666853609675  |
|http://twitpic.com/7iao4 - 